<a href="https://colab.research.google.com/github/super-onigiri-man/ITE3-2/blob/main/FM_%E3%83%99%E3%82%B9%E3%83%88%E3%83%92%E3%83%83%E3%83%88%E3%83%A9%E3%83%B3%E3%82%AD%E3%83%B3%E3%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import requests
from bs4 import BeautifulSoup
import datetime
import pandas as pd

def selecter():

    print("見たい週を選択してください")
    select = int(input("1:今回 2:前回 3:別の回"))
    return select

def OriconTodays():
    # 今日の日付を求める
    d_today = datetime.date.today()
    # 今日の曜日を求める
    todayweek = datetime.date.today().weekday()
    #print(d_today, todayweek)

    #オリコンの発表は毎週水曜日のため、火曜日までは先週のランキングを表示
    #日付は来週の水曜日付となる。
    if (todayweek == 0):  # 今日が月曜日(先週(今週月曜日)のランキング表示)
        Oriconday = d_today
    elif (todayweek == 1):# 今日が火曜日(先週(今週月曜日)のランキング表示)
        Oriconday = d_today - datetime.timedelta(days=1)
    elif (todayweek == 2):# 今日が水曜日
        Oriconday = d_today + datetime.timedelta(days=5)
    elif (todayweek == 3):# 今日が木曜日
        Oriconday = d_today + datetime.timedelta(days=4)
    elif (todayweek == 4):# 今日が金曜日
        Oriconday = d_today + datetime.timedelta(days=3)
    elif (todayweek == 5):# 今日が土曜日
        Oriconday = d_today + datetime.timedelta(days=2)
    elif (todayweek == 6):# 今日が日曜日
        Oriconday = d_today + datetime.timedelta(days=1)

    return Oriconday


def OriconLastWeek():
    # 今日の日付を求める
    d_today = datetime.date.today()
    # 今日の曜日を求める
    todayweek = datetime.date.today().weekday()
    # print(d_today, todayweek)

    # オリコンの発表は毎週水曜日のため、火曜日までは先週のランキングを表示
    # 日付は来週の月曜日付となる。
    if (todayweek == 0):  # 今日が月曜日(先週(今週月曜日)のランキング表示)
        Oriconday = d_today
    elif (todayweek == 1):  # 今日が火曜日(先週(今週月曜日)のランキング表示)
        Oriconday = d_today - datetime.timedelta(days=1)
    elif (todayweek == 2):  # 今日が水曜日
        Oriconday = d_today - datetime.timedelta(days=2)
    elif (todayweek == 3):  # 今日が木曜日
        Oriconday = d_today - datetime.timedelta(days=3)
    elif (todayweek == 4):  # 今日が金曜日
        Oriconday = d_today - datetime.timedelta(days=4)
    elif (todayweek == 5):  # 今日が土曜日
        Oriconday = d_today - datetime.timedelta(days=5)
    elif (todayweek == 6):  # 今日が日曜日
        Oriconday = d_today - datetime.timedelta(days=6)

    return Oriconday


def OriconSelectWeek():
    # 日付の入力を促す
    date = input("2020年8月3日以降の日付を入力してください (YYYY-MM-DD): ")

    # 入力された日付をdatetimeオブジェクトに変換
    dt = datetime.datetime.strptime(date, "%Y-%m-%d")
    dt = dt.date()
    # 曜日を取得
    weekday = dt.weekday()

    # オリコンの発表は毎週水曜日のため、火曜日までは先週のランキングを表示
    # 日付は来週の月曜日付となる。
    if (weekday == 0):  # 今日が月曜日
        Oriconday = dt
    elif (weekday == 1):  # 今日が火曜日
        Oriconday = dt - datetime.timedelta(days=1)
    elif (weekday == 2):  # 今日が水曜日
        Oriconday = dt - datetime.timedelta(days=2)
    elif (weekday == 3):  # 今日が木曜日
        Oriconday = dt - datetime.timedelta(days=3)
    elif (weekday == 4):  # 今日が金曜日
        Oriconday = dt - datetime.timedelta(days=4)
    elif (weekday == 5):  # 今日が土曜日
        Oriconday = dt - datetime.timedelta(days=5)
    elif (weekday == 6):  # 今日が日曜日
        Oriconday = dt - datetime.timedelta(days=6)

    return Oriconday


def OriconWeekRank(Oriconday):#オリコン週間ランキング

    #1位から10位
    load_url = "https://www.oricon.co.jp/rank/js/w/" + str(Oriconday) + "/"
    html = requests.get(load_url)
    soup = BeautifulSoup(html.text, "html.parser")
    links = soup.find(class_="content-rank-main").find_all('h2',class_='title') #曲名
    artist = soup.find(class_="content-rank-main").find_all('p',class_='name') #アーティスト名
    score = 6.0 #独自スコア
    rank = 1 #ランキング
    print(str(Oriconday) + "付けオリコン週間シングルランキング")

    for link, artist in zip(links, artist):

        if rank != 10:#10位以下（１ケタの場合）なら（点数の位置を揃えるため）
            print(" " + str(rank) + "位 " + "{:.1f}　 ".format(score) + link.text + "/" + artist.text)
        else: #10位（２ケタの場合）なら
            print(str(rank) + "位 " + "{:.1f}　 ".format(score) + link.text + "/" + artist.text)
        rank = rank + 1
        score = score - 0.3


    #11位から20位
    load_url = "https://www.oricon.co.jp/rank/js/w/" + str(Oriconday) + "/p/2/"
    html = requests.get(load_url)
    soup = BeautifulSoup(html.text, "html.parser")
    links = soup.find(class_="content-rank-main").find_all('h2', class_='title')  # 曲名
    artist = soup.find(class_="content-rank-main").find_all('p', class_='name')  # アーティスト名
    for link, artist in zip(links, artist):
         print(str(rank) + "位 " + "{:.1f}　 ".format(score) + link.text + "/" + artist.text)
         rank = rank + 1
         score = score - 0.3



def OriconDigitalRank(Oriconday):#オリコンデジタルシングルランキング
    # 1位から10位
    load_url = "https://www.oricon.co.jp/rank/dis/w/" + str(Oriconday) + "/"
    html = requests.get(load_url)
    soup = BeautifulSoup(html.text, "html.parser")
    links = soup.find(class_="content-rank-main").find_all('h2', class_='title')
    artist = soup.find(class_="content-rank-main").find_all('p', class_='name')  # アーティスト名
    rank = 1
    score = float(6.0)
    print(str(Oriconday) + "付けオリコン週間デジタルシングルランキング")
    for link, artist in zip(links, artist):
        if rank != 10:#10位以下（１ケタの場合）なら（点数の位置を揃えるため）
            print(" " + str(rank) + "位 " + "{:.1f}　 ".format(score) + link.text + "/" + artist.text)
        else: #10位（２ケタの場合）なら
            print(str(rank) + "位 " + "{:.1f}　 ".format(score) + link.text + "/" + artist.text)
        rank = rank + 1
        score = score - 0.3

    # 11位から20位
    load_url = "https://www.oricon.co.jp/rank/dis/w/" + str(Oriconday) + "/p/2/"
    html = requests.get(load_url)
    soup = BeautifulSoup(html.text, "html.parser")
    links = soup.find(class_="content-rank-main").find_all('h2', class_='title')
    artist = soup.find(class_="content-rank-main").find_all('p', class_='name')  # アーティスト名
    for link, artist in zip(links, artist):
        print(str(rank) + "位 " + "{:.1f}　 ".format(score) + link.text + "/" + artist.text)
        rank = rank + 1
        score = score - 0.3

def BillboadRank(Oriconday):#ビルボードJAPAN HOT100ランキング

    # オリコンの日付とビルボードの発表日の差を埋めるための計算
    Billday = Oriconday - datetime.timedelta(days=5)

    print(str(Billday) + "付けオリコン週間デジタルシングルランキング")

    #URL(ここを変更すると読み込まなくなります)
    url = 'https://www.billboard-japan.com/charts/detail?a=hot100&year='+str(Oriconday.year)+'&month='+str(Oriconday.month)+'&day='+str(Oriconday.day)
    #URLを取得してくる
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    songs = soup.find_all('p', class_='musuc_title') #曲名
    artists = soup.find_all('p', class_='artist_name') #アーティスト名
    score = 6.0 #基準点

    for i in range(20):#20回繰り替えす
      song = songs[i].text.strip()
      artist = artists[i].text.strip()
      if i < 9:
        print(f" {i+1}位: {format(score, '.1f')} {song} / {artist}") #1位から9位までのランキング
      else:
        print(f"{i+1}位: {format(score, '.1f')} {song} / {artist}") #10位から20位までのランキング
      score = score - 0.3 #scoreを-0.3する


sel = int(selecter())

if sel==1: #今週のランキング表示
    OriconTodays()
    OriconWeekRank(OriconTodays())
    print()
    OriconDigitalRank(OriconTodays())
    print()
    BillboadRank(OriconTodays())

elif sel == 2: #前週のランキング表示
    OriconLastWeek()
    OriconWeekRank(OriconLastWeek())
    print()
    OriconDigitalRank(OriconLastWeek())
    print()
    BillboadRank(OriconLastWeek())

else : #別の週のランキング
    OriconWeekRank(OriconSelectWeek())
    print()
    OriconDigitalRank(OriconSelectWeek())
    print()
    BillboadRank(OriconSelectWeek())


見たい週を選択してください
1:今回 2:前回 3:別の回1
2023-11-27付けオリコン週間シングルランキング
 1位 6.0　 I Wish/なにわ男子
 2位 5.7　 君は何を後悔するのか?/STU48
 3位 5.4　 愛し生きること/MAGIC WORD/King & Prince
 4位 5.1　 あなたがいてくれた/チ・チャンウク
 5位 4.8　 moments(明日に向かって/この場所で)/L(キム・ミョンス)
 6位 4.5　 ナウ・アンド・ゼン/ザ・ビートルズ
 7位 4.2　 R.I.P./ReoNa
 8位 3.9　 TAG ME(HANA_花)/INI
 9位 3.6　 想わせぶりっこ/≠ME
10位 3.3　 霞の向こうへ/中島由貴
11位 3.0　 Prince/Check it/永瀬翔梧
12位 2.7　 素顔のピクセル/スリーズブーケ
13位 2.4　 あのさ、いや別に…/NGT48
14位 2.1　 Wild Party!!/X-BORDER
15位 1.8　 承認欲求/櫻坂46
16位 1.5　 Dangerholic/Snow Man
17位 1.2　 THE IDOLM@STER CINDERELLA GIRLS STARLIGHT MASTER PLATINUM NUMBER 12 Night Time Wander/桐生つかさ(CV:河瀬茉希),大槻唯(CV:山下七海),八神マキノ(CV:二ノ宮ゆい)
18位 0.9　 Black Crack/葛葉
19位 0.6　 01/女王蜂
20位 0.3　 Play The Game/ハイテンション/Ourin-王林-

2023-11-27付けオリコン週間デジタルシングルランキング
 1位 6.0　 唱/Ado
 2位 5.7　 花になって - Be a flower/緑黄色社会
 3位 5.4　 勇者/YOASOBI
 4位 5.1　 Biri-Biri/YOASOBI
 5位 4.8　 unravel/Ado
 6位 4.5　 愛は勝つ/KAN
 7位 4.2　 Monopoly/乃木坂46
 8位 3.9　 SPECIALZ/King Gnu
 9位 3.6　 あのね/あいみょん
10位 3.3　 クラクラ/Ado
11位 3.0　 アイドル/YOASOBI
12

In [24]:
rankings = {}
for i in range(20):
    song = songs[i].text.strip()
    artist = artists[i].text.strip()
    score = score[i].text.strip()
    if song not in rankings:
        rankings[song] = {'artist': artist, 'score': int(score)}
    else:
        rankings[song]['score'] += int(score)

sorted_rankings = sorted(rankings.items(), key=lambda x: x[1]['score'], reverse=True)
for i, (song, info) in enumerate(sorted_rankings[:20]):
    print(f"{i+1}位: {song} - {info['artist']}, スコア: {info['score']}")



NameError: ignored

In [15]:


def BillboadRank(Oriconday):#ビルボードJAPAN HOT100ランキング

    # オリコンの日付とビルボードの発表日の差を埋めるための計算
    Billday = Oriconday - datetime.timedelta(days=5)

    print(str(Billday) + "付けオリコン週間デジタルシングルランキング")

    #URL(ここを変更すると読み込まなくなります)
    url = 'https://www.billboard-japan.com/charts/detail?a=hot100&year='+str(Oriconday.year)+'&month='+str(Oriconday.month)+'&day='+str(Oriconday.day)
    #URLを取得してくる
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    songs = soup.find_all('p', class_='musuc_title') #曲名
    artists = soup.find_all('p', class_='artist_name') #アーティスト名
    score = 6.0 #基準点

    for i in range(20):#20回繰り替えす
      song = songs[i].text.strip()
      artist = artists[i].text.strip()
      if i < 9:
        print(f" {i+1}位: {format(score, '.1f')} {song} / {artist}") #1位から9位までのランキング
      else:
        print(f"{i+1}位: {format(score, '.1f')} {song} / {artist}") #10位から20位までのランキング
      score = score - 0.3 #scoreを-0.3する

OricondayTodays()
BillboadRank(OricondayTodays())


2023-11-22付けオリコン週間デジタルシングルランキング
 1位: 6.0 唱 / Ado
 2位: 5.7 I Wish / なにわ男子
 3位: 5.4 アイドル / YOASOBI
 4位: 5.1 君は何を後悔するのか? / STU48
 5位: 4.8 勇者 / YOASOBI
 6位: 4.5 SPECIALZ / King Gnu
 7位: 4.2 ランデヴー / シャイトープ
 8位: 3.9 怪獣の花唄 / Vaundy
 9位: 3.6 LALALALA / Stray Kids
10位: 3.3 Perfect Night / LE SSERAFIM
11位: 3.0 青のすみか / キタニタツヤ
12位: 2.7 晩餐歌 / tuki.
13位: 2.4 花 / 藤井風
14位: 2.1 HEARTRIS / NiziU
15位: 1.8 花になって / 緑黄色社会
16位: 1.5 I'm a mess / MY FIRST STORY
17位: 1.2 Subtitle / Official髭男dism
18位: 0.9 Seven (feat. Latto) / Jung Kook
19位: 0.6 ケセラセラ / Mrs.GREEN APPLE
20位: 0.3 最高到達点 / SEKAI NO OWARI


IndexError: ignored

In [ ]:


def BillboadRank(Oriconday):#ビルボードJAPAN HOT100ランキング

    # オリコンの日付とビルボードの発表日の差を埋めるための計算
    Billday = Oriconday - datetime.timedelta(days=5)

    print(str(Billday) + "付けオリコン週間デジタルシングルランキング")

    #URL(ここを変更すると読み込まなくなります)
    url = 'https://www.billboard-japan.com/charts/detail?a=hot100&year='+str(Oriconday.year)+'&month='+str(Oriconday.month)+'&day='+str(Oriconday.day)
    #URLを取得してくる
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    songs = soup.find_all('p', class_='musuc_title') #曲名
    artists = soup.find_all('p', class_='artist_name') #アーティスト名
    score = 6.0 #基準点

    for i in range(20):#20回繰り替えす
      song = songs[i].text.strip()
      artist = artists[i].text.strip()
      if i < 9:
        print(f" {i+1}位: {format(score, '.1f')} {song} / {artist}") #1位から9位までのランキング
      else:
        print(f"{i+1}位: {format(score, '.1f')} {song} / {artist}") #10位から20位までのランキング
      score = score - 0.3 #scoreを-0.3する

OricondayTodays()
BillboadRank(OricondayTodays())


2023-11-22付けオリコン週間デジタルシングルランキング
 1位: 6.0 唱 / Ado
 2位: 5.7 I Wish / なにわ男子
 3位: 5.4 アイドル / YOASOBI
 4位: 5.1 君は何を後悔するのか? / STU48
 5位: 4.8 勇者 / YOASOBI
 6位: 4.5 SPECIALZ / King Gnu
 7位: 4.2 ランデヴー / シャイトープ
 8位: 3.9 怪獣の花唄 / Vaundy
 9位: 3.6 LALALALA / Stray Kids
10位: 3.3 Perfect Night / LE SSERAFIM
11位: 3.0 青のすみか / キタニタツヤ
12位: 2.7 晩餐歌 / tuki.
13位: 2.4 花 / 藤井風
14位: 2.1 HEARTRIS / NiziU
15位: 1.8 花になって / 緑黄色社会
16位: 1.5 I'm a mess / MY FIRST STORY
17位: 1.2 Subtitle / Official髭男dism
18位: 0.9 Seven (feat. Latto) / Jung Kook
19位: 0.6 ケセラセラ / Mrs.GREEN APPLE
20位: 0.3 最高到達点 / SEKAI NO OWARI
